Convertir PDF a imagenes de alta calidad con diferentes extenciones

In [8]:
import fitz 
import os

def convert_pdf(pdf_path, output_folder, dpi=600):
    # Extraer el identificador del nombre del archivo PDF (por ejemplo, 'iconos', 'maquinas', 'imagenes')
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    identifier = pdf_name.split(' - ')[-1]

    doc = fitz.open(pdf_path)
    os.makedirs(output_folder, exist_ok=True)

    img_counter = 1  # Contador de imagen

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)

        zoom = dpi / 72 
        mat = fitz.Matrix(zoom, zoom)

        pix = page.get_pixmap(matrix=mat)

        # Formato del nombre de archivo: [identificador]_pag[page_num]_img[img_counter].extensión
        if identifier == "imagenes":
            image_format = "jpg"
        elif identifier == "iconos":
            image_format = "png"
        elif identifier == "maquinas":
            image_format = "jpeg"
        else:
            raise ValueError("Identificador desconocido en el nombre del archivo PDF.")

        output_image_path = os.path.join(output_folder, f"{identifier}_pag{page_num+1}.{image_format}")
        pix.save(output_image_path)

        img_counter += 1  # Incrementar contador de imágenes

    return len(doc)

pdf_files = [
    'instruction_handbook - imagenes.pdf',
    'instruction_handbook - iconos.pdf',
    'instruction_handbook - maquinas.pdf'
]

output_folder = 'imagenes_convertidas'


for pdf_file in pdf_files:
    num_pages = convert_pdf(pdf_file, output_folder)
    print(f"Se han convertido {num_pages} páginas del archivo '{pdf_file}' a imágenes.")


Se han convertido 54 páginas del archivo 'instruction_handbook - imagenes.pdf' a imágenes.
Se han convertido 54 páginas del archivo 'instruction_handbook - iconos.pdf' a imágenes.
Se han convertido 54 páginas del archivo 'instruction_handbook - maquinas.pdf' a imágenes.


Convertir las imagenes a PDF para no perder el orden de las imagenes

In [1]:
from PIL import Image
import os
import re

def sort_key_func(file):
    # Extrae números para ordenar correctamente
    numbers = re.findall(r'\d+', file)
    return [int(num) for num in numbers]

def convert_images_to_pdf(image_folder, output_pdf_path, category):
    # ordena las imágenes por extención
    images = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if category in img and img.endswith((".jpg", ".jpeg", ".png"))]
    images.sort(key=sort_key_func)

    image_list = []

    for img_path in images:
        image = Image.open(img_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image_list.append(image)

    # Guarda las imágenes como un PDF
    if image_list:
        image_list[0].save(output_pdf_path, save_all=True, append_images=image_list[1:], quality=100)

image_folder = 'imagenes_convertidas'

# Categorías de imágenes
categories = ['icono', 'maquina', 'imagen']

# Convertir cada grupo de imágenes a un archivo PDF
for category in categories:
    output_pdf_path = f'pdf_creados\instruction_{category}.pdf'
    convert_images_to_pdf(image_folder, output_pdf_path, category)
    print(f"Las imágenes de la categoría '{category}' han sido convertidas a un PDF en '{output_pdf_path}'.")


Las imágenes de la categoría 'icono' han sido convertidas a un PDF en 'pdf_creados\instruction_icono.pdf'.
Las imágenes de la categoría 'maquina' han sido convertidas a un PDF en 'pdf_creados\instruction_maquina.pdf'.
Las imágenes de la categoría 'imagen' han sido convertidas a un PDF en 'pdf_creados\instruction_imagen.pdf'.


Extraer las imagenes de los PDF

In [5]:
import cv2
import fitz
import numpy as np
import os

def extract_embedded_images(pdf_path, output_folder):
    # Identificar el formato de imagen basado en el nombre del archivo PDF
    if "imagen" in pdf_path:
        image_format = "jpg"
    elif "icono" in pdf_path:
        image_format = "png"
    elif "maquina" in pdf_path:
        image_format = "jpeg"
    else:
        raise ValueError("El nombre del archivo PDF no coincide con ningún criterio establecido.")

    doc = fitz.open(pdf_path)
    os.makedirs(output_folder, exist_ok=True)
    extracted_image_count = 0

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap()
        output_image_path = os.path.join(output_folder, f"page_{page_num+1}.png")
        pix.save(output_image_path)

        # Lee la página
        page_image = cv2.imread(output_image_path)
        gray = cv2.cvtColor(page_image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

        # Encontrar contornos
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        image_index = 0

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w * h > 500:  # umbral
                embedded_image = page_image[y:y+h, x:x+w]
                embedded_image_path = os.path.join(output_folder, f"pag_{page_num+1}_img_{image_index+1}.{image_format}")
                cv2.imwrite(embedded_image_path, embedded_image)
                image_index += 1
                extracted_image_count += 1

    return extracted_image_count

# Lista de archivos PDF para procesar
def find_pdf_files(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.pdf')]
pdf_path = 'pdf_creados' 
pdf_files = find_pdf_files(pdf_path)

# Procesar cada archivo PDF
for pdf_file in pdf_files:
    output_folder = f'imagenes_extraidas_{os.path.splitext(os.path.basename(pdf_file))[0]}'
    num_images = extract_embedded_images(pdf_file, output_folder)
    print(f"Se han extraído {num_images} imágenes incrustadas del archivo '{pdf_file}'.")


Se han extraído 192 imágenes incrustadas del archivo 'pdf_creados\instruction_icono.pdf'.
Se han extraído 77 imágenes incrustadas del archivo 'pdf_creados\instruction_imagen.pdf'.
Se han extraído 41 imágenes incrustadas del archivo 'pdf_creados\instruction_maquina.pdf'.


Convertir las imagenes de la pagina 5 a la 11 a formato JSON en base64

In [ ]:
import json
import base64
import os

# Lista de carpetas donde se encuentran las imágenes
carpetas_imagenes = ['imagenes_extraidas_instruction_icono', 'imagenes_extraidas_instruction_imagen', 'imagenes_extraidas_instruction_maquina']
patrones = ["pag_5", "pag_6", "pag_7", "pag_8", "pag_9", "pag_10", "pag_11"]


for carpeta_imagenes in carpetas_imagenes:
   
    archivos = [f for f in os.listdir(carpeta_imagenes) if any(patron in f for patron in patrones)]

    for archivo in archivos:
        ruta_completa = os.path.join(carpeta_imagenes, archivo)

        # Codificar la imagen en base64
        with open(ruta_completa, mode='rb') as file:
            img = file.read()
            data = base64.b64encode(img).decode('utf-8')

        # Guardar la cadena en un objeto JSON
        json_data = json.dumps(data)

        # Nombre del archivo JSON basado en el nombre de la imagen
        nombre_json = os.path.splitext(archivo)[0] + '.json'
        ruta_json = os.path.join(carpeta_imagenes, nombre_json)

        # Escribir el objeto JSON en un archivo
        with open(ruta_json, 'w') as file:
            file.write(json_data)

    print(f"Conversión completada para la carpeta {carpeta_imagenes}.")
